In [1]:
import torch
import torch.nn as nn
device = "cuda"
device = torch.device(device)

In [2]:
def rotate(word, n):
    return word[n:]+word[0:n]
def shiftRows(state):
    ret = []
    for i in range(4):
        ret += rotate(state[i*4:i*4+4],i)
    return ret
def shiftRowsInv(state):
    ret = []
    for i in range(4):
        ret += rotate(state[i*4:i*4+4],-i)
    return ret

state=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
state = shiftRows(state)
assert state == [1, 2, 3, 4, 6, 7, 8, 5, 11, 12, 9, 10, 16, 13, 14, 15]
state = shiftRowsInv(state)
assert state == [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
print("ok")

ok


In [3]:
import torch.nn.functional as F
noise = 0.0
noise_n = 1

def bin_list(Is):
    ret = []
    for i in Is:
        b0 = torch.tensor([float(a) for a in bin(i)[2:].rjust(8,"0")], dtype=torch.float32)
        # b0 += (torch.FloatTensor(8).uniform_(-noise, +noise))
        b0 = b0.to(device)
        ret = ret + [b0]
    return ret

train_data = []
already = set()

while len(train_data) < 10000:
    Is = torch.randint(0,256,(16,))
    if Is in already:
        continue
    already.add(Is)
    for _ in range(noise_n):
        b = bin_list(Is)
        res = shiftRows(b)
        b = torch.stack(b, dim=0).view(8*16)
        res = torch.stack(res, dim=0).view(8*16)
        train_data.append((b,res))

In [4]:
train_data[0]

(tensor([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
         0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1.,
         0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
         0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
         0., 1.], device='cuda:0'),
 tensor([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0.,
         0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
         0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 

In [5]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

def get_loaders(train_data, device=device):
    test_size = 0.05
    valid_size = 0.05
    batch_size = 500
    num_workers = 0

    #cuda or cpu
    device = torch.device(device)

    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(test_size * num_train))
    split2 = int(np.floor((valid_size+test_size) * num_train))
    train_idx, valid_idx, test_idx = indices[split2:], indices[split:split2], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # prepare data loaders
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
    test_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=test_sampler, num_workers=num_workers)
    return train_loader, valid_loader, test_loader

train_loader, valid_loader, test_loader = get_loaders(train_data)

In [6]:
import numpy as np
def noise_to_int(bits):
    bits = [round(float(b)) for b in bits]
    bits = "".join([str(b) if b in [0,1] else "0" if b<1/10**5 else "1" for b in bits])
    return int(bits,2)

def lin_to_list(t):
    res = []
    for i in range(len(t)//8):
        res += [noise_to_int(t[i*8:i*8+8])]
    return res


In [7]:
X, Y = next(iter(train_loader))

for x,y in zip(X,Y):
    a = lin_to_list(x)
    b = lin_to_list(y)
    b1 = shiftRows(a)
    assert b == b1
print("ok")

ok


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class Norm(nn.Module):
    def __init__(self, num_hidden, eps = 1e-6):
        super().__init__()
    
        self.size = num_hidden
        
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
    
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm


def attention(q, k, v, d_k, mask=None, dropout=None):
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    # print("scores",scores.shape)
    if mask is not None:
        # print("mask",mask.shape)
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
    
    # print("v",v.shape)
    output = torch.matmul(scores, v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, num_hidden, dropout = 0.1):
        super().__init__()
        
        self.num_hidden = num_hidden
        self.d_k = num_hidden // heads
        self.h = heads
        
        self.q_linear = nn.Linear(num_hidden, num_hidden)
        self.v_linear = nn.Linear(num_hidden, num_hidden)
        self.k_linear = nn.Linear(num_hidden, num_hidden)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(num_hidden, num_hidden)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into N heads
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * N * sl * num_hidden
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        

        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.num_hidden)
        output = self.out(concat)
    
        return output

class FeedForward(nn.Module):
    def __init__(self, input_num, output_num, d_ff=2048, dropout = 0.1):
        super().__init__() 
    
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(input_num, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, output_num)
    
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [9]:
import torch.nn.functional as F

class ShiftRows(nn.Module):
    def __init__(self):
        super(ShiftRows, self).__init__()
        
        self.body = nn.Sequential(
            nn.Linear(8*16,8*16),
        )
        
    def forward(self, x):
        out = self.body(x)
        return out

In [10]:
def train(save_file, model, criterion, train_loader, valid_loader, optimizer=None, n_epochs = 100000, f=lin_to_list, lrate=0.005):
    # number of epochs to train the model

    if optimizer is None:
        # specify optimizer (stochastic gradient descent) and learning rate = 0.001
        optimizer = torch.optim.Adam(model.parameters(), lr=lrate)#, weight_decay=0.00000001)

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf # set initial "min" to infinity
    
    for epoch in range(n_epochs):
        # monitor training loss
        train_loss = 0.0
        valid_loss = 0.0
        results = 0
        results_n = 0
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        i=0
        for X, target in train_loader:
            i+=1
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            target = target.to(device)
            output = model(X)
            # calculate the loss
            # print(output)
            # print(target)
            loss = criterion(output, target) #
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update running training loss
            train_loss += loss.item()*X.size(0)
            if epoch%100 == 0:
                for x,y in zip(output,target):
                    # print(x.cpu().detach().numpy(),y)
                    a = f(x.cpu().detach().numpy())
                    # a = int(x[0])
                    b = f(y.cpu().detach().numpy())
                    # b = int(y[0])
                    # a = noise_to_int(x)
                    # b = noise_to_int(y)
                    
                    
                    # print(a,b)
                    # print(float(x[0]),float(y[0]))
                    if a==b:
                        
                        results +=1
                    results_n+=1
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        for X, target in valid_loader:
        
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # target = target.to(device)
            # calculate the loss
            loss = criterion(output, target)
            # update running validation loss
            valid_loss += loss.item()*X.size(0)
            

        # print training/validation statistics
        # calculate average loss over an epoch
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch+1,
            train_loss,
            valid_loss
            ))

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            torch.save(model.state_dict(), save_file)
            valid_loss_min = valid_loss
            if train_loss <= 0.000000001:
                print("stop: loss <= 0.00000")
                return
            else:
                print(" loss >= 0.00000")
        
        if results_n != 0 :
            print(f"{results/results_n=}")
            print(f"{results}")
            # if results == results_n and valid_loss <= valid_loss_min:
            #     print("stop: no errors")
            #     return
        

In [12]:
# model = ShiftRows().to(device)
criterion = nn.MSELoss()
train("shiftrows.pt", model, criterion, train_loader, valid_loader, f=lin_to_list, lrate=0.0005)

Epoch: 1 	Training Loss: 0.000151 	Validation Loss: 0.000001
Validation loss decreased (inf --> 0.000001).  Saving model ...
 loss >= 0.00000
results/results_n=1.0
9000
Epoch: 2 	Training Loss: 0.000021 	Validation Loss: 0.000000
Validation loss decreased (0.000001 --> 0.000000).  Saving model ...
 loss >= 0.00000
Epoch: 3 	Training Loss: 0.000004 	Validation Loss: 0.000000
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
 loss >= 0.00000
Epoch: 4 	Training Loss: 0.000001 	Validation Loss: 0.000000
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
 loss >= 0.00000
Epoch: 5 	Training Loss: 0.000000 	Validation Loss: 0.000000
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
 loss >= 0.00000
Epoch: 6 	Training Loss: 0.000000 	Validation Loss: 0.000000
Validation loss decreased (0.000000 --> 0.000000).  Saving model ...
 loss >= 0.00000
Epoch: 7 	Training Loss: 0.000000 	Validation Loss: 0.000000
Validation loss decreased (0.000000

In [22]:
model = ShiftRows().to(device)
model.load_state_dict(torch.load("shiftrows.pt"))

<All keys matched successfully>

In [23]:
model.eval()
X, Y = next(iter(train_loader))

# print(X,Y)
results = 0
results_n = 0
O = model(X)
for x,y in zip(O,Y):
    a = lin_to_list(x.cpu().detach().numpy())
    b = lin_to_list(y.cpu().detach().numpy())
    # b = int(y[0]) pos_to_int
    # b = noise_to_int(y)
    # print(x,y)
    # print(a,b)
    if a==b:
        
        results +=1
    results_n +=1
print(f"{results/results_n=}")

results/results_n=1.0


In [24]:
model.eval()
X, Y = next(iter(train_loader))

# print(X,Y)
results = 0
results_n = 0
O = X
n = 1000
for i in range(n):
    O = model(O)
for x,y in zip(X,O):
    # print(x,y)
    a = lin_to_list(x.cpu().detach().numpy())
    b = lin_to_list(y.cpu().detach().numpy())
    # b = int(y[0]) pos_to_int
    # b = noise_to_int(y)
    
    r = a
    for i in range(n):
        r = shiftRows(r)
    # print(r,b)
    if r==b:
        
        results +=1
    results_n +=1
print(f"after {n} mix columns")
print(f"{results/results_n=}")
# print(model.state_dict())

after 1000 mix columns
results/results_n=1.0


In [21]:
import torch
state = torch.load("shiftrows.pt")
for w,b in zip(state['body.0.weight'], state['body.0.bias']):
    a = w+b
    for i,v in enumerate(a):
        if v > 0.9:
            w[i] = 1.0
        else:
            w[i] = 0.0
for i in range(len(state["body.0.bias"])):
    state["body.0.bias"][i] = 0.0
torch.save(state,"shiftrows.pt")